In [ ]:
import pandas as pd
from utils.load_dataframe import load_dataframe

In [ ]:
orders = load_dataframe('orders')
reviews = load_dataframe('order_reviews')
products = load_dataframe('products')

display(orders.dtypes)
display(reviews.dtypes)
display(products.dtypes)


# Com a função load_dataframe fica de forma automática os tipos de cada coluna, e agora não precisa ficar convertendo a cada novo arquivo criado. Basta chama-la que as colunas ficarão com os tipos corretos.

In [ ]:
# Evolução de pedidos ao longo do tempo

# Importação dos datasets
